In [57]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords

In [58]:
pf.head()

,Comments,label
0,How to stop the war Stop the media escalating it,1
1,WW1 : History Books WW2 : History Books WW3 : ...,1
2,Let the politicians go to war and soldiers tak...,1
3,We'll be praying for the peace of everyone.......,1
4,We are praying that this won't go too far. Ble...,1


In [41]:
pf=pd.read_csv("iran.csv")

In [47]:
pf['Comments']


0       How to stop the war Stop the media escalating it
1      WW1 : History Books WW2 : History Books WW3 : ...
2      Let the politicians go to war and soldiers tak...
3      We'll be praying for the peace of everyone.......
4      We are praying that this won't go too far. Ble...
                             ...                        
521                           Sound effects ruined this.
522    A cat riding a Saket board now I seen almost e...
523    wtf are this sound effects, they're annoying, ...
524    I hate how there is annoying sound effects in ...
525    The laughing and sound effects are terrible no...
Name: Comments, Length: 526, dtype: object

In [43]:
pf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 526 entries, 0 to 525
Data columns (total 2 columns):
Comments    526 non-null object
label       526 non-null int64
dtypes: int64(1), object(1)
memory usage: 8.3+ KB


In [44]:
pf.describe()

,label
count,526.000000
mean,0.492395
std,0.500418
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [45]:
pf.groupby("label").describe()

Comments                                                               
         count unique                                                top freq
label                                                                        
0          267    260                                         Cute. Cats    2
1          259    257  I hope that the leaders Love for his people (c...    2

In [19]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [48]:
pf['Comments'].apply(text_process)

0                   [stop, war, Stop, media, escalating]
1      [WW1, History, Books, WW2, History, Books, WW3...
2      [Let, politicians, go, war, soldiers, take, re...
3      [Well, praying, peace, everyonefrom, Philippines]
4      [praying, wont, go, far, Blessings, everyone, ...
                             ...                        
521                             [Sound, effects, ruined]
522    [cat, riding, Saket, board, seen, almost, ever...
523    [wtf, sound, effects, theyre, annoying, cats, ...
524                [hate, annoying, sound, effects, Ugh]
525    [laughing, sound, effects, terrible, offense, ...
Name: Comments, Length: 526, dtype: object

In [49]:
from sklearn.feature_extraction.text import CountVectorizer

In [50]:

bow_transformer = CountVectorizer(analyzer=text_process).fit(pf['Comments'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

1974


In [52]:
pf_bow = bow_transformer.transform(pf['Comments'])

In [53]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(pf_bow)

In [55]:
pf_tfidf = tfidf_transformer.transform(pf_bow)

In [56]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(pf_tfidf, pf['label'])

In [60]:
predictions = model.predict(pf_tfidf)

In [61]:
from sklearn.metrics import classification_report
print (classification_report(pf['label'], predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       267
           1       0.99      0.98      0.99       259

    accuracy                           0.99       526
   macro avg       0.99      0.99      0.99       526
weighted avg       0.99      0.99      0.99       526



In [66]:
from sklearn.model_selection import train_test_split

pf_train, pf_test, label_train, label_test = \
train_test_split(pf['Comments'],pf['label'], test_size=0.2)

In [65]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [67]:
pipeline.fit(pf_train,label_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x000001824963FA68>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [68]:
predictions = pipeline.predict(pf_test)

In [69]:
print(classification_report(predictions,label_test))

              precision    recall  f1-score   support

           0       0.91      0.86      0.88        56
           1       0.85      0.90      0.87        50

    accuracy                           0.88       106
   macro avg       0.88      0.88      0.88       106
weighted avg       0.88      0.88      0.88       106

